In [1]:
import re
import xgboost
import numpy as np
import pandas as pd
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [2]:
def myInt(myList):
    return map(int, myList)

In [3]:
df=pd.read_csv("Data/train.tsv",delimiter='\t')
df.columns=[''.join(col.split()) for col in df.columns]
print df.columns
df.head()

Index([u'item_id', u'Seller', u'Actors', u'ActualColor', u'ArtistID',
       u'AspectRatio', u'Color', u'GenreID', u'ISBN', u'ItemClassID',
       u'LiteraryGenre', u'MPAARating', u'ProductLongDescription',
       u'ProductName', u'ProductShortDescription', u'Publisher',
       u'RecommendedLocation', u'RecommendedRoom', u'RecommendedUse',
       u'ShortDescription', u'Synopsis', u'actual_color', u'tag'],
      dtype='object')


,item_id,Seller,Actors,ActualColor,ArtistID,AspectRatio,Color,GenreID,ISBN,ItemClassID,...,ProductName,ProductShortDescription,Publisher,RecommendedLocation,RecommendedRoom,RecommendedUse,ShortDescription,Synopsis,actual_color,tag
0,0,Circuit City,NaN,Multicolor,NaN,NaN,NaN,NaN,NaN,1,...,Cyberpower Metered ATS PDU 200-240V 20A 1U 10-...,NaN,NaN,NaN,NaN,NaN,"The CyberPower PDU20MHVT10AT, a 10-outlet 1U r...",NaN,NaN,[4483]
1,1,NaN,NaN,Black,NaN,NaN,NaN,NaN,NaN,1,...,Peerless SmartMount SF650P Universal Flat Wall...,Peerless SmartMount SF650P Universal Flat Wall...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[581514]
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,...,"DAYTON 31EH66 Control Transformer Fused,4.02H ...",NaN,NaN,NaN,NaN,NaN,short description is not available,NaN,NaN,[4483]
3,3,Pcrush,NaN,Multicolor,NaN,NaN,NaN,NaN,NaN,19,...,STEREN-CUSTOM INSTALL 255-047 STEREN MINI TO R...,NaN,NaN,NaN,NaN,NaN,<p>Steren 3.5mm to RCA Y-Cable 255-047</p><p>1...,NaN,NaN,[4537]
4,4,Walmart.com,NaN,Black,NaN,NaN,NaN,NaN,NaN,19,...,"LG 47LN5700 47"" 1080p 120Hz LED (3.17"" ultra-s...",NaN,NaN,NaN,NaN,NaN,"The LG 42LN5700 47"" 1080p 120Hz LED HDTV is a ...",NaN,NaN,"[1229817, 1229821]"


In [4]:
descriptions=['ProductName','ProductShortDescription','ProductLongDescription','ShortDescription']
df=df.replace(np.nan,'null')
df['isbook']=0
df['ismovie']=0
df['isTV']=0
df['text']=''
df=df.replace('short description is not available','null')
df.ISBN=df.ISBN.replace('null',0)
for i, row in df.iterrows():
    if row.Synopsis != 'null' or row.MPAARating != 'null' or row.Actors != 'null':
        df.set_value(i,'ismovie',1)
    if row.ISBN > 0 or row.Publisher != 'null' or row.LiteraryGenre != 'null':
        df.set_value(i,'isbook',1)
    if row.AspectRatio != 'null':
        df.set_value(i,'isTV',1)
    buf=''
    for desc in descriptions:
        if row[desc] != 'null':
            buf+=re.sub('<[^>]*>',' ',row[desc])+' '
    if buf.split():
        df.set_value(i,'text',buf.lower())
    else:
        df.set_value(i,'text','None')

df['others']=(df.isbook | df.isTV | df.ismovie)
df.others=~df.others.astype(bool)
df.others=df.others.astype(int)

dfF=df[['item_id','text','ItemClassID','ismovie','isbook','isTV','others','tag']]
dfF=dfF[dfF.text!='None']
dfF.tag=dfF.tag.apply(lambda x: x[1:len(x)-1].split(','))
dfF.tag=map(myInt,dfF.tag)
dfF.head()

,item_id,text,ItemClassID,ismovie,isbook,isTV,others,tag
0,0,cyberpower metered ats pdu 200-240v 20a 1u 10-...,1,0,0,0,1,[4483]
1,1,peerless smartmount sf650p universal flat wall...,1,0,0,0,1,[581514]
2,2,"dayton 31eh66 control transformer fused,4.02h ...",1,0,0,0,1,[4483]
3,3,steren-custom install 255-047 steren mini to r...,19,0,0,0,1,[4537]
4,4,"lg 47ln5700 47"" 1080p 120hz led (3.17"" ultra-s...",19,0,0,0,1,"[1229817, 1229821]"


In [5]:
dfT=pd.read_csv("Data/test.tsv",delimiter='\t')
dfT.columns=[''.join(col.split()) for col in dfT.columns]
dfT=dfT.replace(np.nan,'null')
dfT['isbook']=0
dfT['ismovie']=0
dfT['isTV']=0
dfT['text']=''
dfT=dfT.replace('short description is not available','null')
dfT.ISBN=dfT.ISBN.replace('null',0)
for i, row in dfT.iterrows():
    if row.Synopsis != 'null' or row.MPAARating != 'null' or row.Actors != 'null':
        dfT.set_value(i,'ismovie',1)
    if row.ISBN > 0 or row.Publisher != 'null' or row.LiteraryGenre != 'null':
        dfT.set_value(i,'isbook',1)
    if row.AspectRatio != 'null':
        dfT.set_value(i,'isTV',1)
    buf=''
    for desc in descriptions:
        if row[desc] != 'null':
            buf+=re.sub('<[^>]*>',' ',row[desc])+' '
    if buf.split():
        dfT.set_value(i,'text',buf.lower())
    else:
        dfT.set_value(i,'text','None')
l=LabelEncoder()
ItemClassIDs=l.fit_transform(dfF.ItemClassID.append(dfT.ItemClassID))
dfF.ItemClassID=ItemClassIDs[:dfF.shape[0]]
dfT.ItemClassID=ItemClassIDs[dfF.shape[0]:]
dfT['others']=(dfT.isbook | dfT.isTV | dfT.ismovie)
dfT.others=~dfT.others.astype(bool)
dfT.others=dfT.others.astype(int)
dfTF=dfT[['item_id','text','ItemClassID','ismovie','isbook','isTV','others']]
dfTF.head()

,item_id,text,ItemClassID,ismovie,isbook,isTV,others
0,10593,"peerless suf651 tv mount, 37-75 in ultra-thin,...",0,0,0,0,1
1,10594,100 pcs toothed flexible marker cable zip wire...,0,0,0,0,1
2,10595,house hunters renovation: down under to over t...,21,1,0,0,0
3,10596,traveler3usb surge suppressor notebook 3 outle...,0,0,0,0,1
4,10597,75ft rapidrun runner multi-format cmg rapidrun...,2,0,0,0,1


In [6]:
vect=TfidfVectorizer(stop_words='english',ngram_range=(1,2),min_df=2)
X=vect.fit_transform(dfF.text)
X=X.toarray()

In [7]:
cols=['ItemClassID','ismovie','isbook','isTV','others']
X=np.hstack((X,np.array(dfF[cols]).reshape(dfF.shape[0],5)))

In [8]:
labbin=MultiLabelBinarizer()
Y=labbin.fit_transform(list(dfF.tag))

In [9]:
clf=OneVsRestClassifier(LinearSVC(C=1.1,class_weight='balanced'))
clf.fit(X,Y)

OneVsRestClassifier(estimator=LinearSVC(C=1.1, class_weight='balanced', dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
          n_jobs=1)

In [13]:
X_test=vect.transform(dfTF.text)
X_test=X_test.toarray()
X_test=np.hstack((X_test,np.array(dfTF[cols]).reshape(dfTF.shape[0],5)))

In [14]:
preds=clf.predict(X_test)
preds=labbin.inverse_transform(preds)
preds=[list(p) for p in preds]
preds=map(myInt, preds)

In [15]:
dfTF['tag']=preds
op=dfTF[['item_id','tag']]
op.to_csv('result.tsv',index=False,sep='\t')

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
